In [170]:
from SimPy.Simulation import *
import random
import numpy as np
import math
import read_data as rd
import scipy
import scipy.stats as st

In [171]:
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper

In [172]:
filenames = ["/Users/viviandong/Desktop/COMP312/DATA304-project-main/Data Collection/KevinYeData.txt",
             "/Users/viviandong/Desktop/COMP312/DATA304-project-main/Data Collection/PatrickQData.txt",
             "/Users/viviandong/Desktop/COMP312/DATA304-project-main/Data Collection/TamaHoareData.txt",
             "/Users/viviandong/Desktop/COMP312/DATA304-project-main/Data Collection/dataviviandong.txt"]
results = rd.ReadData(filenames=filenames)
AllArrivals = results[0]
AllServices = results[1]

In [173]:
#From our analysis, it suggests that Pearson3 is the best in approximating ‘inter-arrival time’ Data.
# Pearson3 Distribution Parameters
skew,loc,scale = st.pearson3.fit(AllArrivals)

In [174]:
# For our 'service times' data, it suggests that beta distribution is the best fit for the data.
# Beta Distribution Parameters
beta_a, beta_b,loc,scale = st.beta.fit(AllServices)

In [175]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = abs(st.pearson3.rvs(skew))
            yield hold, self, t

In [176]:
class Arrival(Process):
    """an arrival"""
    n=0 

    def run(self):
        Arrival.n +=1
        
        arrivetime = now()
        
        G.numbermon.observe(Arrival.n)
        
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
        yield request, self, G.server
        yield hold, self, np.random.beta(beta_a, beta_b)
        yield release, self, G.server
        
        Arrival.n -=1
        
        G.numbermon.observe(Arrival.n)
        
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
        delay = now()-arrivetime
        G.delaymon.observe(delay)

In [177]:
class G:
    server = 'dummy'
    delaymon = 'Monitor' # W
    numbermon = "Monitor" # L
    busymon = 'Monitor' # B

In [178]:
def model(N, maxtime, rvseed,K):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(K)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()

    Arrival.n = 0
    
    # simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    
    return W, L, B

The LAB Cafe is open from 7.30am to 3pm a total of 7.5 hours per day which is equivalent to 27000 seconds which would be the maxtime of each simulation.

In [183]:
## Experiment ----------------


for Kap in [1,2,3]:
    allW = []
    allL = []
    allB = []
    allLambdaEffective = []

    for k in range(50):
        seed = 123*k
        result = model(N=10000, maxtime=27000, rvseed=seed, K=Kap)
        allW.append(result[0])
        allL.append(result[1])
        allB.append(result[2])
        allLambdaEffective.append(result[1]/result[0])


    print("=================")
    print("When K=",Kap)
    print("    Estimate of W:", np.mean(allW))
    print("    Conf int of W:", conf(allW))
    print("    Estimate of L:", np.mean(allL))
    print("    Conf int of L:", conf(allL))
    print("    Estimate of B:", np.mean(allB))
    print("    Conf int of B:", conf(allB))
    print("    Estimate of LambdaEffective:", np.mean(allLambdaEffective))
    print("    Conf int of LambdaEffective:", conf(allLambdaEffective))

When K= 1
    Estimate of W: 0.09080309307981936
    Conf int of W: (0.09061532826307689, 0.09099085789656183)
    Estimate of L: 0.12795803870205108
    Conf int of L: (0.1275736949645114, 0.12834238243959076)
    Estimate of B: 0.1214983155757442
    Conf int of B: (0.12114326143780896, 0.12185336971367944)
    Estimate of LambdaEffective: 1.4092118802993747
    Conf int of LambdaEffective: (1.4051998449365513, 1.4132239156621982)
When K= 2
    Estimate of W: 0.08653149948885601
    Conf int of W: (0.08633931141002504, 0.08672368756768697)
    Estimate of L: 0.12175499491676887
    Conf int of L: (0.12134014793873144, 0.12216984189480631)
    Estimate of B: 0.11788256006260335
    Conf int of B: (0.1174816575124894, 0.1182834626127173)
    Estimate of LambdaEffective: 1.4070589482940934
    Conf int of LambdaEffective: (1.403466973434821, 1.4106509231533657)
When K= 3
    Estimate of W: 0.08627299571992532
    Conf int of W: (0.0861189071077185, 0.08642708433213214)
    Estimate of L